In [ ]:
from yoloface.face_detector import YoloDetector
import numpy as np
from PIL import Image
import cv2

model = YoloDetector(target_size=720, device="cuda:0", min_face=5)
orgimg = np.array(Image.open('faces/22.jpg'))
bboxes,points = model.predict(orgimg)

# Рисование bounding boxes
image = orgimg.copy()
for bbox in bboxes[0]:
    xmin, ymin, xmax, ymax = bbox
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

print(bboxes[0])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.imshow(image)
plt.show()

In [ ]:
# check
import paddle
paddle.utils.run_check()

# confirm the paddle's version
!python -c "import paddle; print(paddle.__version__)"

In [ ]:
import sys
sys.path.append("/workspace/yoloface/")

In [ ]:
import cv2
import torch
import pandas as pd
from yoloface.face_detector import YoloDetector
from PIL import Image
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort

# Model
face_detector = YoloDetector(target_size=720, device="cuda:0", min_face=80)

# Путь к видеофайлу
video_path = 'mp4/Video_example.mp4'  # Замените путь на ваш видеофайл

# Загрузка видео
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Tracker preset:

# Embeder parameters:
# EMBEDDER_CHOICES = [
#     "mobilenet",
#     "torchreid",
#     "clip_RN50",
#     "clip_RN101",
#     "clip_RN50x4",
#     "clip_RN50x16",
#     "clip_ViT-B/32",
#     "clip_ViT-B/16",
# ]

tracker_params = {
    "max_iou_distance": 0.7,
    "max_age": 30,
    "n_init": 3,
    "nms_max_overlap": 1.0,
    "max_cosine_distance": 0.2,
    "nn_budget": None,
    "gating_only_position": False,
    "override_track_class": None,
    "embedder": "mobilenet",
    "half": True,
    "bgr": True,
    "embedder_gpu": True,
    "embedder_model_name": None,
    "embedder_wts": None,
    "polygon": False,
    "today": None,
}

# Tracker
tracker = DeepSort(**tracker_params)

# Создание объекта VideoWriter для записи результата
output_path = 'output.mp4'  # Замените путь по своему усмотрению
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
 
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
        
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Детекция объектов
    bboxes, points = face_detector.predict(frame)
    
    # bboxes in update_tracks expected to be a list of detections, each in tuples of ( [left,top,w,h], confidence, detection_class )
    bboxes_formatted = []
    for xmin, ymin, xmax, ymax in bboxes[0]:
        bboxes_formatted.append(([xmin, ymax, xmax - xmin, ymax - ymin], 1.0, 'face'))
        
    frame_with_boxes = frame.copy()
    tracks = tracker.update_tracks(bboxes_formatted, frame=frame) 
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        left, top, right, bottom = track.to_ltrb()
        cv2.rectangle(frame_with_boxes, (left, bottom), (right, top), (0, 255, 0), 2)    
        
    output.write(frame_with_boxes)

# Освобождение ресурсов
cap.release()
output.release()
cv2.destroyAllWindows()

In [ ]:
# == Display results ==
from IPython.display import HTML
from base64 import b64encode

video_path = "output.mp4"

mp4 = open(video_path, "rb").read()
data_url = "data:video/webm;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=600 controls>
      <source src="{data_url}" type="video/webm">
</video>
""")